# Codex predicted solutions

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

GENERATED_ROOT_PATH = os.path.join("..", "input", "bugnet")
GENERATED_PAIRS_PATH = os.path.join(GENERATED_ROOT_PATH, "generated_pairs.csv")
CODEX_PAIRS_PATH = os.path.join(GENERATED_ROOT_PATH, "codex_results.csv")

In [2]:
codex_df = pd.read_csv(CODEX_PAIRS_PATH, keep_default_na=False)

codex_df.head()

,problem_id,language,original_status,original_src,changed_src,change,line,error,codex_predicted
0,p00001,Python,Runtime Error,num = [int(input()) for i in range(10)]\nnum.s...,num = [int(input()) for i in range(10)]\nnum.s...,replace,2,TypeError,for i in range(3):\n\n# Q: What is the purpos...
1,p00001,C++,Runtime Error,#include <bits/stdc++.h>\nusing namespace std;...,#include <bits/stdc++.h>\nusing namespace std;...,replace,4,-6,vector<int> mountain(10);
2,p00001,C++,Runtime Error,#include <algorithm>\n#include <iostream>\nusi...,#include <algorithm>\n#include <iostream>\nusi...,replace,9,-11,for (int i = 9; i >= 7; i--)
3,p00001,C++,Time Limit Exceeded,#include <stdio.h>\nint main(void) {\n int a ...,#include <stdio.h>\nint main(void) {\n int a ...,replace,3,TLE,"while (scanf(""%d"", &n) == 1)"
4,p00001,C++,Runtime Error,"#include <stdio.h>\nint main(void) {\n int i,...","#include <stdio.h>\nint main(void) {\n int i,...",replace,4,-11,"scanf(""%d"", &n[i]);"


In [3]:
codex_df["line_str"] = codex_df.apply(lambda row: row["changed_src"].splitlines()[row["line"]], axis="columns")

def codex_to_line_str(codex_predicted: str) -> str:
    codex_predicted = codex_predicted.strip()
    codex_lines = codex_predicted.splitlines()
    if len(codex_lines) > 0:
        codex_predicted = codex_lines[0]
    return codex_predicted

codex_df["codex_line_str"] = codex_df.apply(lambda row: codex_to_line_str(row["codex_predicted"]), axis="columns")

codex_df["correct"] = codex_df.apply(lambda row: row["line_str"] == row["codex_line_str"], axis="columns")

correct = codex_df["correct"].sum()
print(f"The accuracy of the codex api is {correct / len(codex_df)}")

The accuracy of the codex api is 0.054


In [4]:
codex_lang_df = codex_df.groupby("language")["correct"].agg(["sum", "count"])
codex_lang_df["accuracy"] = codex_lang_df["sum"] / codex_lang_df["count"]

codex_lang_df

,sum,count,accuracy
language,,,
C++,0,500,0.000
Python,54,500,0.108


In [5]:
def color_source(source_code: str, line: int, color: str='red'):
    lines = source_code.splitlines(keepends=True)
    
    text = ""
    for i, line_str in enumerate(lines):
        for char in line_str:
            norm_color = 'black'
            if char == ' ':
                char = "•"
                norm_color = 'lightgrey'
            if char == '\n':
                char = "↵\n"
                norm_color = 'lightgrey'
            text += f'<span style="color:{color if i == line else norm_color};">{char}</span>'

    return "<pre>" + text + "</pre>"

In [6]:
codex_df["original_src_html"] = codex_df.apply(
    lambda row: color_source(row["original_src"], row["line"], color="red"), 
    axis="columns",
)

codex_df["changed_src_html"] = codex_df.apply(
    lambda row: color_source(row["changed_src"], row["line"], color="green"), 
    axis="columns",
)

codex_df["codex_src_html"] = codex_df.apply(
    lambda row: color_source(row["codex_line_str"], 0, color="green" if row["correct"] else "red"), 
    axis="columns",
)

In [7]:
codex_correct_df = codex_df[codex_df["correct"]]
for i in range(min(len(codex_correct_df), 10)):
    display(HTML(f"<h2>Example {i}</h2>"))
    
    display(HTML(f"<h6>Original Source Code</h6>"))
    display(HTML(codex_correct_df["original_src_html"].iloc[i]))

    display(HTML(f"<h6>Changed Source Code</h6>"))
    display(HTML(codex_correct_df["changed_src_html"].iloc[i]))

    display(HTML(f"<h6>Codex Prediction</h6>"))
    display(HTML(codex_correct_df["codex_src_html"].iloc[i]))

In [8]:
for i in range(10):
    display(HTML(f"<h2>Example {i}</h2>"))
    
    display(HTML(f"<h6>Original Source Code</h6>"))
    display(HTML(codex_df["original_src_html"].iloc[i]))

    display(HTML(f"<h6>Changed Source Code</h6>"))
    display(HTML(codex_df["changed_src_html"].iloc[i]))

    display(HTML(f"<h6>Codex Prediction</h6>"))
    display(HTML(codex_df["codex_src_html"].iloc[i]))